## Adapatation of CMEMS data download provided by Favien
Seems far more proffesional and clean and also fixes my hardcod problem


In [30]:
import os
import copernicusmarine as cpm
from pathlib import Path
import ast
from datetime import datetime
import shutil

In [31]:

def create_missing_directories():
    # Define the path to the parent directory
    parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

    # Check if 'data' folder exists in the parent directory
    data_dir = os.path.join(parent_dir, 'data')
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)
        print("'data' folder created in the parent directory.")
    else:
        print("'data' folder already exists in the parent directory.")

    # Check if 'satellite' directory exists inside 'data' folder
    satellite_dir = os.path.join(data_dir, 'satellite')
    if not os.path.exists(satellite_dir):
        os.makedirs(satellite_dir)
        print("'satellite' directory created inside 'data' folder.")
    else:
        print("'satellite' directory already exists inside 'data' folder.")

if __name__ == "__main__":
    create_missing_directories()



'data' folder already exists in the parent directory.
'satellite' directory already exists inside 'data' folder.


In [32]:
datasets_to_extract = [
    "cmems_obs-sst_glo_phy_nrt_l4_P1D-m",  # daily L4 SST (near-real-time)
    "cmems_obs-oc_glo_bgc-optics_my_l4-multi-4km_P1M"]          # bbp


# Directory setup
parent_dir = os.path.abspath(os.path.join(os.getcwd()))
data_dir = "../data"
satellite_dir = "../data/satellite"

# Date and time setup
start_time = "2021-01-01T00:00:00"  # Format time as YYYY-MM-DD"T"HH:MM:SS
now = datetime.now()
end_time = now.strftime("2024-12-31T00:00:00")
end_time2 = now.strftime("20241231")
print(end_time2)

print(parent_dir)
print(data_dir)
print(satellite_dir)

20241231
/Users/davidlewis/Desktop/Masters/scripts
../data
../data/satellite


In [33]:
# Geographical bounds
min_depth = 0
max_depth = 10000
min_lon = -40
max_lon = -10
min_lat = 50
max_lat = 80

# Loop through each dataset
for dataset_to_extract in datasets_to_extract:
    # Dynamically set the output filename based on the dataset name
    saved_netCDF_as = f"CMEMS_{dataset_to_extract}_2021-2024.nc"
    filepath = os.path.join(satellite_dir, saved_netCDF_as)

    # Check if the file already exists
    if os.path.exists(filepath):
        response = input(f"Do you want to remove and overwrite the file \"{saved_netCDF_as}\" (y/n): ").strip().lower()
        if response == 'y':
            os.remove(filepath)
            print(f"{filepath} has been removed.")
            print(f"{saved_netCDF_as}. Proceeding to download.")
        else:
            print("Download canceled.")
            continue
    else:
        print(f"{saved_netCDF_as} does not exist. Proceeding to download.")
    
    # Perform the data subset operation
    cpm.subset(
        dataset_id=dataset_to_extract,
        minimum_longitude=min_lon,
        maximum_longitude=max_lon,
        minimum_latitude=min_lat,
        maximum_latitude=max_lat,
        start_datetime=start_time,
        end_datetime=end_time,
        minimum_depth=min_depth,
        maximum_depth=max_depth,
        output_filename=saved_netCDF_as,
        output_directory=satellite_dir
    )

CMEMS_cmems_obs-sst_glo_phy_nrt_l4_P1D-m_2021-2024.nc does not exist. Proceeding to download.


INFO - 2025-09-04T09:27:50Z - Selected dataset version: "202303"
INFO - 2025-09-04T09:27:50Z - Selected dataset part: "default"
WARNING - 2025-09-04T09:27:50Z - Some of your subset selection [50, 80] for the latitude dimension exceed the dataset coordinates [-79.94999694824219, 79.94999694824219]
INFO - 2025-09-04T09:27:51Z - Starting download. Please wait...
100%|██████████| 84/84 [04:56<00:00,  3.53s/it]  
INFO - 2025-09-04T09:32:47Z - Successfully downloaded to ../data/satellite/CMEMS_cmems_obs-sst_glo_phy_nrt_l4_P1D-m_2021-2024.nc


CMEMS_cmems_obs-oc_glo_bgc-optics_my_l4-multi-4km_P1M_2021-2024.nc does not exist. Proceeding to download.


INFO - 2025-09-04T09:32:49Z - Selected dataset version: "202311"
INFO - 2025-09-04T09:32:49Z - Selected dataset part: "default"
INFO - 2025-09-04T09:32:49Z - Starting download. Please wait...
100%|██████████| 165/165 [02:11<00:00,  1.26it/s]
INFO - 2025-09-04T09:35:01Z - Successfully downloaded to ../data/satellite/CMEMS_cmems_obs-oc_glo_bgc-optics_my_l4-multi-4km_P1M_2021-2024.nc
